In [3]:
# summary result

import pandas as pd
import os
from statistics import mean,stdev
files = os.listdir('./train_emp_attr_davis2')
total = {}
def stat_res(data):
    m = format(mean(data),'.4f')
    dev = format(stdev(data),'.4f')
    return f'{m}({dev})'

for i in range(0,len(files),5):
    this_result = []
    aggr = dict()
    aggr['test_auc'] = []
    aggr['test_logloss'] = []
    aggr['test_aupr']=[]
    aggr['test_recall'] = []
    name = files[i]
    for j in range(5):
        with open('./train_emp_attr_davis2/'+files[i+j]) as f:
            a = f.readlines()
            res = []
            for epoch in a:
                if epoch!='\n':
                    each = epoch.split(',')
                    each[-1] = each[-1].replace('\n','')
                    res.append(each)
            if res:
                if res[-1][0].split(':')[0]=='epoch':
                    this_result.append(res[  int(res[-1][0].split(':')[1]) ])
                else:
                    this_result.append(res[-1])
    for ind in this_result:
        try:
            aggr['test_auc'].append(float(ind[2].split(':')[1].split('/')[1]  ))
        except:
            print(ind)
            print(this_result)
            print(i)
        aggr['test_logloss'].append(float(ind[3].split(':')[1].split('/')[1]  ))
        aggr['test_aupr'].append(float(ind[6].split(':')[1].split('/')[1]  ))
        aggr['test_recall'].append(float(ind[10].split(':')[1].split('/')[1]  ))
    total[name] = aggr
for key in total.keys():
    total[key]['test_auc'] = stat_res(total[key]['test_auc'])
    total[key]['test_logloss'] = stat_res(total[key]['test_logloss'])
    total[key]['test_aupr'] = stat_res(total[key]['test_aupr'])
    total[key]['test_recall'] = stat_res(total[key]['test_recall'])
with open('summary.txt','w') as f:
    for key in total.keys():
        f.write(key)
        f.write('\n')
        f.write(str(total[key]))
        f.write('\n\n')

In [4]:
# show similarity

import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import seaborn as sns

n_davis = 68 + 379
n_kiba = 1997+228
model = torch.load('./model_final_KIBA/corss 0 on split0/model_checkpoint.pth')
model['feature_embedding.weight'].size()

plt.figure(figsize=(12,10), dpi= 80)
feature_set = model['feature_embedding.weight'][n_kiba:]
fdrug_set , ftarget_set = feature_set[125:174].cpu(),feature_set[174:-2].cpu()
matrix = cosine_similarity(fdrug_set,ftarget_set)
sns.heatmap(matrix, xticklabels = False, yticklabels = False, cmap='coolwarm', center=0, annot=False).set(xlabel = "Target Bridge Node", ylabel = "Drug Bridge Node")

plt.title('Bridge Node Similarity on KIBA', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('./simi.png')
plt.show()